In [29]:
import pandas as pd
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from fairlearn.metrics import MetricFrame, selection_rate, count, demographic_parity_difference
from ipynb.fs.defs import ERED

### Load Data

In [30]:
# Overall data
train_data = pd.read_pickle(open("sentence-transformers/train_emb.pkl", "rb"))
dev_data = pd.read_pickle(open("sentence-transformers/dev_emb.pkl", "rb"))
# train_data = pd.read_pickle(open("tfidf/train_tfidf.pkl", "rb"))
# dev_data = pd.read_pickle(open("tfidf/dev_tfidf.pkl", "rb"))

train_X = list(train_data['TFIDF'])
train_y = (train_data['Sentiment'] == 'positive') * 1

dev_X = list(dev_data['TFIDF'])
dev_y  = (dev_data['Sentiment'] == 'positive') * 1

# AAE
# Filter and extract all AAE data
AAE_train_data = train_data[train_data['Demographic'] == 'AAE']
AAE_train_X = list(AAE_train_data['TFIDF'])
AAE_train_y = (AAE_train_data['Sentiment'] == 'positive') * 1

AAE_dev_data = dev_data[dev_data['Demographic'] == 'AAE']
AAE_dev_X = list(AAE_dev_data['TFIDF'])
AAE_dev_y  = (AAE_dev_data['Sentiment'] == 'positive') * 1

# SAE
# Filter and extract all SAE data
SAE_train_data = train_data[train_data['Demographic'] == 'SAE']
SAE_train_X = list(SAE_train_data['TFIDF'])
SAE_train_y = (SAE_train_data['Sentiment'] == 'positive') * 1

SAE_dev_data = dev_data[dev_data['Demographic'] == 'SAE']
SAE_dev_X = list(SAE_dev_data['TFIDF'])
SAE_dev_y  = (SAE_dev_data['Sentiment'] == 'positive') * 1

### Train Model

In [31]:
nb = GaussianNB()
nb.fit(train_X, train_y)

GaussianNB()

### Prediction

#### 1. Overall

In [32]:
overall_labels_predict = nb.predict(dev_X)
overall_confusion = confusion_matrix(dev_y, overall_labels_predict)
print(classification_report(dev_y, overall_labels_predict))
print(overall_confusion)

              precision    recall  f1-score   support

           0       0.60      0.70      0.64      2000
           1       0.64      0.53      0.58      2000

    accuracy                           0.61      4000
   macro avg       0.62      0.61      0.61      4000
weighted avg       0.62      0.61      0.61      4000

[[1398  602]
 [ 939 1061]]


#### 2. AAE

In [33]:
AAE_labels_predict = nb.predict(AAE_dev_X)
AAE_confusion = confusion_matrix(AAE_dev_y, AAE_labels_predict)
print(classification_report(AAE_dev_y, AAE_labels_predict))
print(AAE_confusion)

              precision    recall  f1-score   support

           0       0.57      0.62      0.59      1000
           1       0.58      0.54      0.56      1000

    accuracy                           0.58      2000
   macro avg       0.58      0.58      0.58      2000
weighted avg       0.58      0.58      0.58      2000

[[617 383]
 [462 538]]


#### 3. SAE

In [34]:
SAE_labels_predict = nb.predict(SAE_dev_X)
SAE_confusion = confusion_matrix(SAE_dev_y, SAE_labels_predict)
print(classification_report(SAE_dev_y, SAE_labels_predict))
print(SAE_confusion)

              precision    recall  f1-score   support

           0       0.62      0.78      0.69      1000
           1       0.70      0.52      0.60      1000

    accuracy                           0.65      2000
   macro avg       0.66      0.65      0.65      2000
weighted avg       0.66      0.65      0.65      2000

[[781 219]
 [477 523]]


### Bias Evaluation

#### 1. Accuracy + Demographic count

In [35]:
multi_metrics = {'accuracy': accuracy_score, 'count': count}
am = MetricFrame(metrics=multi_metrics, y_true=dev_y, y_pred=overall_labels_predict, sensitive_features=dev_data['Demographic'])
print(am.overall)
print(am.by_group)

accuracy    0.61475
count          4000
dtype: object
            accuracy count
Demographic               
AAE           0.5775  2000
SAE            0.652  2000


#### 2. Selection rate + Demographic parity difference

In [36]:
sm = MetricFrame(metrics=selection_rate, y_true=dev_y, y_pred=overall_labels_predict, sensitive_features=dev_data['Demographic'])
print(sm.overall)
print(sm.by_group)

group_metrics = demographic_parity_difference(y_true=dev_y, y_pred=overall_labels_predict, sensitive_features=dev_data['Demographic'])
print("\nDifference between selection rate")
print("Demographic parity difference: {}".format(round(group_metrics, 3)))

0.41575
Demographic
AAE    0.4605
SAE     0.371
Name: selection_rate, dtype: object

Difference between selection rate
Demographic parity difference: 0.09


#### 3. Error Rate Equality Difference

In [37]:
matrix_lst = [AAE_confusion, SAE_confusion]
ERED.cal_sum_ered(overall_confusion, matrix_lst)

Sum of FPR difference: 0.164
Sum of FNR difference: 0.015
Sum of Error Rate Equality Difference: 0.179


0.17899999999999996

---

## Bias Mitigation

### Exponentiated Gradient

In [38]:
from fairlearn.reductions import ExponentiatedGradient, DemographicParity
np.random.seed(0)

In [39]:
constraint = DemographicParity()
classifier = GaussianNB()
mitigator = ExponentiatedGradient(classifier, constraints=constraint)
mitigator.fit(np.array(train_X), train_y, sensitive_features=train_data['Demographic'])

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/fairlearn/reductions/_moments/utility_parity.py:251: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  lambda_event = (lambda_vec["+"] - self.ratio * lambda_vec["-"]).sum(level=_EVENT) / \
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/fairlearn/reductions/_moments/utility_parity.py:251: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  lambda_event = (lambda_vec["+"] - self.ratio * lambda_vec["-"]).sum(level=_EVENT) / \
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/fairlearn/reductions/_moments/utility_parity.py:251: FutureWarning

ExponentiatedGradient(constraints=<fairlearn.reductions._moments.utility_parity.DemographicParity object at 0x144cc2440>,
                      estimator=GaussianNB(), nu=0.0012151603391080718)

### Prediction

#### 1. Overall

In [40]:
overall_labels_predict = mitigator.predict(dev_X)
overall_confusion = confusion_matrix(dev_y, overall_labels_predict)
print(classification_report(dev_y, overall_labels_predict))
print(overall_confusion)

              precision    recall  f1-score   support

           0       0.59      0.71      0.65      2000
           1       0.64      0.51      0.57      2000

    accuracy                           0.61      4000
   macro avg       0.62      0.61      0.61      4000
weighted avg       0.62      0.61      0.61      4000

[[1423  577]
 [ 981 1019]]


#### 2. AAE

In [41]:
AAE_labels_predict = mitigator.predict(AAE_dev_X)
AAE_confusion = confusion_matrix(AAE_dev_y, AAE_labels_predict)
print(classification_report(AAE_dev_y, AAE_labels_predict))
print(AAE_confusion)

              precision    recall  f1-score   support

           0       0.56      0.67      0.61      1000
           1       0.60      0.48      0.53      1000

    accuracy                           0.58      2000
   macro avg       0.58      0.58      0.57      2000
weighted avg       0.58      0.58      0.57      2000

[[674 326]
 [521 479]]


#### 3. SAE

In [42]:
SAE_labels_predict = mitigator.predict(SAE_dev_X)
SAE_confusion = confusion_matrix(SAE_dev_y, SAE_labels_predict)
print(classification_report(SAE_dev_y, SAE_labels_predict))
print(SAE_confusion)

              precision    recall  f1-score   support

           0       0.62      0.77      0.69      1000
           1       0.70      0.53      0.60      1000

    accuracy                           0.65      2000
   macro avg       0.66      0.65      0.65      2000
weighted avg       0.66      0.65      0.65      2000

[[770 230]
 [467 533]]


### Bias Evaluation

#### 1. Accuracy + Demographic count

In [43]:
multi_metrics = {'accuracy': accuracy_score, 'count': count}
am = MetricFrame(metrics=multi_metrics, y_true=dev_y, y_pred=overall_labels_predict, sensitive_features=dev_data['Demographic'])
print(am.overall)
print(am.by_group)

accuracy    0.6105
count         4000
dtype: object
            accuracy count
Demographic               
AAE           0.5665  2000
SAE           0.6545  2000


#### 2. Selection rate + Demographic parity difference

In [44]:
sm = MetricFrame(metrics=selection_rate, y_true=dev_y, y_pred=overall_labels_predict, sensitive_features=dev_data['Demographic'])
print(sm.overall)
print(sm.by_group)

group_metrics = demographic_parity_difference(y_true=dev_y, y_pred=overall_labels_predict, sensitive_features=dev_data['Demographic'])
print("\nDifference between selection rate")
print("Demographic parity difference: {}".format(round(group_metrics, 3)))

0.399
Demographic
AAE    0.4115
SAE    0.3865
Name: selection_rate, dtype: object

Difference between selection rate
Demographic parity difference: 0.025


#### 3. Error Rate Equality Difference

In [45]:
matrix_lst = [AAE_confusion, SAE_confusion]
ERED.cal_sum_ered(overall_confusion, matrix_lst)

Sum of FPR difference: 0.096
Sum of FNR difference: 0.054
Sum of Error Rate Equality Difference: 0.15


0.15